<a href="https://colab.research.google.com/github/cjstntjd/youtube_emotion_analysis/blob/master/word.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/gdrive',force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive


In [0]:
import numpy as np 
import pandas as pd

data = pd.read_csv('/gdrive/My Drive/사가시오 개발 자료/댓글 처리 관련 코드/data.csv',encoding='CP949')

In [0]:
data.head(10)

,Unnamed: 0,ID,댓글
0,0,yeondukong연두콩,"??CC click please??????발렌타이 데이에,, 핸드워시 영상 올리는 ..."
1,1,프린스송,손도 깨끗이 씻고좋은 일에도 쓰이고일석 이조네요??여윽시 연두콩님??p.s 연두콩님...
2,2,blueming,ㅠㅠ올라온거 확인하자마자 사라져서 내 손가락 원망하고 있었는데..다행히 빨리 다시 ...
3,3,은비,생일 축하합니다~ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ손 씻으면서 속으...
4,4,쥬닝,코로나까지 걱정해 주는 언니...?? 언니도 조심하세여
5,5,김팥쥐PPATJi,2:09 생활공작소 진짜 좋아영!! 다른 제품도 거의다 써봤는데 케이스도 예쁘고 넘...
6,6,Jiwony,라이브 때 ??생활 밀착형 유투버??가 핸드워시 소개해준다구해서 기다렷구만유~~~~...
7,7,해바라기,아니 마음도 너무 이쁘고 오늘 얼구도 넘 이쁘고ㅠㅠ 진짜 화장품이랑 옷 정보 너무 ...
8,8,수수,언니 머리 흑발 염색한거에여 아니면 원래 머리에용?? 예뻐요 ????
9,9,민혀뀨,두콩언니 얼굴 무슨일이예여??? 너무 존예잖아요!!!!!!


#한글 자연어 처리 시작
코랩에 없는 3파티 목록 : konlpy 

pip install 다운

In [0]:
!pip install konlpy

In [0]:
import scipy as sp
import pandas as pd
import numpy as np
import scipy as sp

from konlpy.tag import Kkma; kkma = Kkma()
from konlpy.tag import Hannanum; hannanum = Hannanum()
from konlpy.tag import Okt; t = Okt()

from wordcloud import WordCloud, STOPWORDS
from PIL import Image
import platform
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from bs4 import BeautifulSoup
from urllib.request import urlopen
import urllib
import time

from tqdm import tqdm_notebook
import nltk
from nltk.tokenize import word_tokenize

import gensim
from gensim.models import word2vec


#자연어 처리 기초 

##꼬꼬마

In [0]:
from konlpy.tag import Kkma
kkma = Kkma()

In [0]:
kkma.sentences('안녕하세요 모바일시스템공학 정영환 입니다!!!')


['안녕하세요', '모바일시스템공학 정 영 환 입니다!!!']

In [0]:
kkma.nouns('안녕하세요 모바일시스템공학 정영환 입니다!!!')

['안녕', '모바일', '모바일시스템공학', '시스템', '공학', '정', '정영환', '영', '환']

In [0]:
kkma.pos('안녕하세요 모바일시스템공학 정영환 입니다!!!')

[('안녕', 'NNG'),
 ('하', 'XSV'),
 ('세요', 'EFN'),
 ('모바일', 'NNG'),
 ('시스템', 'NNG'),
 ('공학', 'NNG'),
 ('정', 'NNG'),
 ('영', 'NNG'),
 ('환', 'NNG'),
 ('이', 'VCP'),
 ('ㅂ니다', 'EFN'),
 ('!!!', 'SW')]

###사전에 없는 정영환 이라는 이름 형태의 단어를 인식하는데는 어려움이 있음 

유튜브에서 실질적으로 사용 되는 은어 형태의 밈을 데이터화 시켜야 하는가? 아니면 이것을 자의 적으로 처리해 줄수 있는 데이터 셋이 존재 하는가 ?  

좀 더 눌러보자


okt를 활용한 데이터 처리 시작

In [0]:
from konlpy.tag import Okt
t = Okt()

In [0]:
t.nouns('안녕하세요 모바일시스템공학 정영환 입니다!!!')

['모바일', '시스템', '공학', '정영환']

아주 고무적임 

꼬꼬마에서는 인식하지 못했던 정영환 이라는 이름을 정확하게 인식 했고 띄어쓰기가 되지 않은 상태의 모바일 시스템 공학이라는 단어도 정확하게 의미 별로 뜯어 놓음 

거기에 의미 없는 인사에서 안녕은 아예 명사 처리도 되지 않음 

자체 데이터 셋의 필터링 기능이 훌륭함 

여기에서 드는 의문점

그렇다면 문법에 맞지 않은 띄어 쓰기도 정확하게 처리 가능한가?

은어도 명사처리가 되서 튀어나올수 있나??

은어형태의 수식어도 수식어 처리를 하는가?

라이브러리의 한계점을 정확하게 파악해야 해당 부분을 처리 가능

In [0]:
t.nouns('안녕하세요 모바일시스템공학 정영환 입니다. 앙기모찌~~')

['모바일', '시스템', '공학', '정영환', '기모찌']

In [0]:
t.nouns('안녕하세요 모바일시스템공학 정영환 입니다. 앙기모띠~~')

['모바일', '시스템', '공학', '정영환', '띠']

1. 은어 형태를 넣어 봄 앙을 수식어 처리 하는 기능까지 있음 

In [0]:
t.nouns('안녕하세요 모바일시스템공학정영환 잆니니다')

['모바일', '시스템', '공학', '정영환', '잆니니']

2. 문법에 맞지 않는 단어는 명사 처리 함
3. 띄어쓰기를 하지 않아도 의미 단위로 끊어서 명사처리 하며 여기서도 이름형태는 정확하게 처리 

In [0]:
t.nouns('아니 우리정글뭐하냐고 용타이밍인데 무슨레드스틸을 하고있어 리얼 롤알못 ㅇㅈ? ㅇㅇㅈ~')

['우리', '정글', '뭐', '용', '타이밍', '레드', '스틸', '리얼', '롤']

아니를 수식어 처리함 굉장한 기능 

우리 정글 뭐하냐고 에서 가장 중요한 단어는 정글 정확하게 찝어냄 
롤알못과 같은 은어 파악에 어려움이 있음 해당 데이터는 조금 더 언어처리 해줄 필요가 있음

해당 문장을 형태소 분석 해보자 

In [0]:
t.pos('아니 우리정글뭐하냐고 용타이밍인데 무슨레드스틸을 하고있어 리얼 롤알못 ㅇㅈ? ㅇㅇㅈ~')

[('아니', 'Adjective'),
 ('우리', 'Noun'),
 ('정글', 'Noun'),
 ('뭐', 'Noun'),
 ('하냐고', 'Verb'),
 ('용', 'Noun'),
 ('타이밍', 'Noun'),
 ('인데', 'Josa'),
 ('무슨', 'Modifier'),
 ('레드', 'Noun'),
 ('스틸', 'Noun'),
 ('을', 'Josa'),
 ('하고있어', 'Verb'),
 ('리얼', 'Noun'),
 ('롤', 'Noun'),
 ('알못', 'Verb'),
 ('ㅇㅈ', 'KoreanParticle'),
 ('?', 'Punctuation'),
 ('ㅇㅇㅈ', 'KoreanParticle'),
 ('~', 'Punctuation')]

일반적으로 못 이라는 단어는 동사에 포함됨 그래서 분류기는 롤은 단어 알못은 동사를 분류함 

그런데 어원적으로 생각하면 롤을 알지도 못하는 사람에서 동사가 맞기도 함 

그럼 롤알못 자체를 부정 단어 처리 할게 아니라 못이 들어간 범위 자체를 부정 처리 하는게 옳음 



ㅇㅈ? ㅇㅇㅈ~ 과도 같은 작은 친구들을 치환해주는 작업이 필요함 



In [0]:
#나이브베이즈 분류기의 이해 (한글)

from konlpy.tag import Okt

In [0]:
pos_tagger = Okt()

In [0]:
train = [('손도 깨끗이 씻고좋은 일에도 쓰이고일석 이조네요??여윽시 연두콩님??p.s 연두콩님도 코로나 조심하세요??','pos'),('2:09 생활공작소 진짜 좋아영!! 다른 제품도 거의다 써봤는데 케이스도 예쁘고 넘 좋음 ??','pos'),('두콩언니 얼굴 무슨일이예여??? 너무 존예잖아요!!!!!!','pos'),('왜 삭제됐던 거져 ㅠㅠ','neg'),('엄마 여기 무서워.......조회수는 1인데 좋아요가 4개야..흑흑흐규ㅠ','neg'),('아 지짜 우리언니 개귀여어 ㅠㅠ 판검사 드립 진짜 기여어 미치겐네...','pos'),('핸드크림 진짜 향기 존좋...ㅠㅠㅠ 진심 저것만 쓰는중인데 여기서 보니 반갑네유 ㅋㅋㅋ','pos'),('infp 내적 친밀감ㅋㅋㅋㅋㅋㅋㅋ 막막했는데 추천템 영상 만들어주셔서 감사해요??','pos'),('아녀와세여여어붕!!ㅋㅋㅋㅋㅋㅋㅋㅋ너무귀여워요 언니ㅜㅜㅠㅠㅠㅠㅠ','pos')]

In [0]:
train

[('손도 깨끗이 씻고좋은 일에도 쓰이고일석 이조네요??여윽시 연두콩님??p.s 연두콩님도 코로나 조심하세요??', 'pos'),
 ('2:09 생활공작소 진짜 좋아영!! 다른 제품도 거의다 써봤는데 케이스도 예쁘고 넘 좋음 ??', 'pos'),
 ('두콩언니 얼굴 무슨일이예여??? 너무 존예잖아요!!!!!!', 'pos'),
 ('왜 삭제됐던 거져 ㅠㅠ', 'neg'),
 ('엄마 여기 무서워.......조회수는 1인데 좋아요가 4개야..흑흑흐규ㅠ', 'neg'),
 ('아 지짜 우리언니 개귀여어 ㅠㅠ 판검사 드립 진짜 기여어 미치겐네...', 'pos'),
 ('핸드크림 진짜 향기 존좋...ㅠㅠㅠ 진심 저것만 쓰는중인데 여기서 보니 반갑네유 ㅋㅋㅋ', 'pos'),
 ('infp 내적 친밀감ㅋㅋㅋㅋㅋㅋㅋ 막막했는데 추천템 영상 만들어주셔서 감사해요??', 'pos'),
 ('아녀와세여여어붕!!ㅋㅋㅋㅋㅋㅋㅋㅋ너무귀여워요 언니ㅜㅜㅠㅠㅠㅠㅠ', 'pos')]

In [0]:
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [0]:
all_words = set(word.lower() for sentence in train
                                 for word in word_tokenize(sentence[0]))
all_words

{'!',
 '...',
 '.조회수는',
 '1인데',
 '2:09',
 '4개야..흑흑흐규ㅠ',
 '?',
 'infp',
 'p.s',
 'ㅋㅋㅋ',
 'ㅋㅋㅋㅋㅋㅋㅋㅋ너무귀여워요',
 'ㅠㅠ',
 'ㅠㅠㅠ',
 '감사해요',
 '개귀여어',
 '거의다',
 '거져',
 '기여어',
 '깨끗이',
 '내적',
 '너무',
 '넘',
 '다른',
 '두콩언니',
 '드립',
 '막막했는데',
 '만들어주셔서',
 '무서워',
 '무슨일이예여',
 '미치겐네',
 '반갑네유',
 '보니',
 '삭제됐던',
 '생활공작소',
 '손도',
 '써봤는데',
 '쓰는중인데',
 '쓰이고일석',
 '씻고좋은',
 '아',
 '아녀와세여여어붕',
 '언니ㅜㅜㅠㅠㅠㅠㅠ',
 '얼굴',
 '엄마',
 '여기',
 '여기서',
 '여윽시',
 '연두콩님',
 '연두콩님도',
 '영상',
 '예쁘고',
 '왜',
 '우리언니',
 '이조네요',
 '일에도',
 '저것만',
 '제품도',
 '조심하세요',
 '존예잖아요',
 '존좋',
 '좋아영',
 '좋아요가',
 '좋음',
 '지짜',
 '진심',
 '진짜',
 '추천템',
 '친밀감ㅋㅋㅋㅋㅋㅋㅋ',
 '케이스도',
 '코로나',
 '판검사',
 '핸드크림',
 '향기'}

In [0]:
t = [({word: (word in word_tokenize(x[0])) for word in all_words}, x[1])
                                           for x in train]
t

[({'!': False,
   '...': False,
   '.조회수는': False,
   '1인데': False,
   '2:09': False,
   '4개야..흑흑흐규ㅠ': False,
   '?': True,
   'infp': False,
   'p.s': True,
   'ㅋㅋㅋ': False,
   'ㅋㅋㅋㅋㅋㅋㅋㅋ너무귀여워요': False,
   'ㅠㅠ': False,
   'ㅠㅠㅠ': False,
   '감사해요': False,
   '개귀여어': False,
   '거의다': False,
   '거져': False,
   '기여어': False,
   '깨끗이': True,
   '내적': False,
   '너무': False,
   '넘': False,
   '다른': False,
   '두콩언니': False,
   '드립': False,
   '막막했는데': False,
   '만들어주셔서': False,
   '무서워': False,
   '무슨일이예여': False,
   '미치겐네': False,
   '반갑네유': False,
   '보니': False,
   '삭제됐던': False,
   '생활공작소': False,
   '손도': True,
   '써봤는데': False,
   '쓰는중인데': False,
   '쓰이고일석': True,
   '씻고좋은': True,
   '아': False,
   '아녀와세여여어붕': False,
   '언니ㅜㅜㅠㅠㅠㅠㅠ': False,
   '얼굴': False,
   '엄마': False,
   '여기': False,
   '여기서': False,
   '여윽시': True,
   '연두콩님': True,
   '연두콩님도': True,
   '영상': False,
   '예쁘고': False,
   '왜': False,
   '우리언니': False,
   '이조네요': True,
   '일에도': True,
   '저것만': False,
   '제품도': False,
   '

In [0]:
def tokenize(doc):
    return ['/'.join(t) for t in pos_tagger.pos(doc, norm=True, stem=True)]

In [0]:
train_docs = [(tokenize(row[0]), row[1]) for row in train]
train_docs

[(['손/Noun',
   '도/Josa',
   '깨끗이/Adverb',
   '씻다/Verb',
   '좋다/Adjective',
   '일/Noun',
   '에도/Josa',
   '쓰이다/Verb',
   '고일석/Noun',
   '이조/Noun',
   '네/Suffix',
   '요/Josa',
   '??/Punctuation',
   '여윽시/Noun',
   '연/Modifier',
   '두/Modifier',
   '콩/Noun',
   '님/Suffix',
   '??/Punctuation',
   'p/Alpha',
   './Punctuation',
   's/Alpha',
   '연/Modifier',
   '두/Modifier',
   '콩/Noun',
   '님/Suffix',
   '도/Josa',
   '코로나/Noun',
   '조심하다/Verb',
   '??/Punctuation'],
  'pos'),
 (['2:09/Number',
   '생활/Noun',
   '공작/Noun',
   '소/Noun',
   '진짜/Noun',
   '좋다/Adjective',
   '아영/Noun',
   '!!/Punctuation',
   '다른/Noun',
   '제품/Noun',
   '도/Josa',
   '거의/Noun',
   '다/Josa',
   '써다/Verb',
   '보다/Verb',
   '케이스/Noun',
   '도/Josa',
   '예쁘다/Adjective',
   '넘다/Verb',
   '좋다/Adjective',
   '??/Punctuation'],
  'pos'),
 (['두/Determiner',
   '콩/Noun',
   '언니/Suffix',
   '얼굴/Noun',
   '무슨/Modifier',
   '일이/Noun',
   '예여/Josa',
   '???/Punctuation',
   '너무/Adverb',
   '존예/Noun',
   '잖다/Verb',
   '!!!!!!

In [0]:
tokens = [t for d in train_docs for t in d[0]]
tokens

['손/Noun',
 '도/Josa',
 '깨끗이/Adverb',
 '씻다/Verb',
 '좋다/Adjective',
 '일/Noun',
 '에도/Josa',
 '쓰이다/Verb',
 '고일석/Noun',
 '이조/Noun',
 '네/Suffix',
 '요/Josa',
 '??/Punctuation',
 '여윽시/Noun',
 '연/Modifier',
 '두/Modifier',
 '콩/Noun',
 '님/Suffix',
 '??/Punctuation',
 'p/Alpha',
 './Punctuation',
 's/Alpha',
 '연/Modifier',
 '두/Modifier',
 '콩/Noun',
 '님/Suffix',
 '도/Josa',
 '코로나/Noun',
 '조심하다/Verb',
 '??/Punctuation',
 '2:09/Number',
 '생활/Noun',
 '공작/Noun',
 '소/Noun',
 '진짜/Noun',
 '좋다/Adjective',
 '아영/Noun',
 '!!/Punctuation',
 '다른/Noun',
 '제품/Noun',
 '도/Josa',
 '거의/Noun',
 '다/Josa',
 '써다/Verb',
 '보다/Verb',
 '케이스/Noun',
 '도/Josa',
 '예쁘다/Adjective',
 '넘다/Verb',
 '좋다/Adjective',
 '??/Punctuation',
 '두/Determiner',
 '콩/Noun',
 '언니/Suffix',
 '얼굴/Noun',
 '무슨/Modifier',
 '일이/Noun',
 '예여/Josa',
 '???/Punctuation',
 '너무/Adverb',
 '존예/Noun',
 '잖다/Verb',
 '!!!!!!/Punctuation',
 '왜/Noun',
 '삭제/Noun',
 '돼다/Verb',
 '거/Noun',
 '지다/Verb',
 'ㅠㅠ/KoreanParticle',
 '엄마/Noun',
 '여기/Noun',
 '무섭다/Adjective',
 '......./P

In [0]:
def term_exists(doc):
    return {word: (word in set(doc)) for word in tokens}

In [0]:
train_xy = [(term_exists(d), c) for d,c in train_docs]
train_xy

[({'!!!!!!/Punctuation': False,
   '!!/Punctuation': False,
   '......./Punctuation': False,
   '.../Punctuation': False,
   '../Punctuation': False,
   './Punctuation': True,
   '1/Number': False,
   '2:09/Number': False,
   '4/Number': False,
   '??/Punctuation': True,
   '???/Punctuation': False,
   'infp/Alpha': False,
   'p/Alpha': True,
   's/Alpha': True,
   'ㅋㅋㅋ/KoreanParticle': False,
   'ㅜㅜㅠ/KoreanParticle': False,
   'ㅠ/KoreanParticle': False,
   'ㅠㅠ/KoreanParticle': False,
   'ㅠㅠㅠ/KoreanParticle': False,
   '감/Noun': False,
   '감사하다/Verb': False,
   '개/Noun': False,
   '개다/Verb': False,
   '거/Noun': False,
   '거의/Noun': False,
   '것/Noun': False,
   '고일석/Noun': True,
   '공작/Noun': False,
   '귀엽다/Adjective': False,
   '깨끗이/Adverb': True,
   '내적/Noun': False,
   '너무/Adverb': False,
   '넘다/Verb': False,
   '네/Determiner': False,
   '네/Josa': False,
   '네/Suffix': True,
   '는/Josa': False,
   '님/Suffix': True,
   '다/Josa': False,
   '다른/Noun': False,
   '도/Josa': True,
   '돼다/V

In [0]:
classifier = nltk.NaiveBayesClassifier.train(train_xy)

In [0]:
test_sentence = [("엄마 여기 무서워.......조회수는 1인데 좋아요가 4개야..흑흑흐규ㅠ")]

In [0]:
test_docs = pos_tagger.pos(test_sentence[0])
test_docs

[('엄마', 'Noun'),
 ('여기', 'Noun'),
 ('무서워', 'Adjective'),
 ('.......', 'Punctuation'),
 ('조', 'Modifier'),
 ('회수', 'Noun'),
 ('는', 'Josa'),
 ('1', 'Number'),
 ('인데', 'Josa'),
 ('좋아', 'Adjective'),
 ('요가', 'Noun'),
 ('4', 'Number'),
 ('개야', 'Verb'),
 ('..', 'Punctuation'),
 ('흑흑흐규', 'Noun'),
 ('ㅠ', 'KoreanParticle')]

In [0]:
test_sent_features = {word: (word in tokens) for word in test_docs}
test_sent_features

{('..', 'Punctuation'): False,
 ('.......', 'Punctuation'): False,
 ('1', 'Number'): False,
 ('4', 'Number'): False,
 ('ㅠ', 'KoreanParticle'): False,
 ('개야', 'Verb'): False,
 ('는', 'Josa'): False,
 ('무서워', 'Adjective'): False,
 ('엄마', 'Noun'): False,
 ('여기', 'Noun'): False,
 ('요가', 'Noun'): False,
 ('인데', 'Josa'): False,
 ('조', 'Modifier'): False,
 ('좋아', 'Adjective'): False,
 ('회수', 'Noun'): False,
 ('흑흑흐규', 'Noun'): False}

In [0]:
classifier.classify(test_sent_features)

'pos'

In [0]:
from sklearn.feature_extraction.text import CountVectorizer

In [0]:
vectorizer = CountVectorizer(min_df = 1)

In [0]:
contents=[]

for x in data['댓글']:
  contents.append(x)

In [0]:
contents

['??CC click please??????발렌타이 데이에,, 핸드워시 영상 올리는 내가 일류입니다,,,여러분?? 속으로 노래부르며 30초 손씻기 잊지마유?? 생활밀착형 유튜버니께 생활밀착형 콘텐츠로 찾아뵐게유????',
 '손도 깨끗이 씻고좋은 일에도 쓰이고일석 이조네요??여윽시 연두콩님??p.s 연두콩님도 코로나 조심하세요??',
 'ㅠㅠ올라온거 확인하자마자 사라져서 내 손가락 원망하고 있었는데..다행히 빨리 다시 올라와서 오늘 하루도 행복??',
 '생일 축하합니다~ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ손 씻으면서 속으로 신나게 부르고 있을 상상을 하니 너무 귀엽고 신박해 ㅠ??',
 '코로나까지 걱정해 주는 언니...?? 언니도 조심하세여',
 '2:09 생활공작소 진짜 좋아영!! 다른 제품도 거의다 써봤는데 케이스도 예쁘고 넘 좋음 ??',
 '라이브 때 ??생활 밀착형 유투버??가 핸드워시 소개해준다구해서 기다렷구만유~~~~넘 고마워유~~~~??',
 '아니 마음도 너무 이쁘고 오늘 얼구도 넘 이쁘고ㅠㅠ 진짜 화장품이랑 옷 정보 너무 궁그매여!!',
 '언니 머리 흑발 염색한거에여 아니면 원래 머리에용?? 예뻐요 ????',
 '두콩언니 얼굴 무슨일이예여??? 너무 존예잖아요!!!!!!',
 '거의 끝까지 다 봤는데 그래도 다시 볼께요^^이번주 주말에 날씨가 비가온뒤에 일요일에는 눈이 온다고 하니 길 다니실때 조심하세요^^',
 '비행기에서 빵터졌지 뭐에유ㅋㅋㅋ 이런 꿀 정보 감사해유 증말루 두콩님두 코로나 조심하세유~~',
 '유익한 영상 항상 감사히 봐요 ?? 감사함은 댓글로 종종 갚을게요 ????',
 '이 언니??너무 스윗하고 상큼하잖아유.. 첫멘트로 그냥 아주 녹아내렸습니다??',
 '여러분 세상편한 발씻기 저거 진짜 소확행이에요 발 씻으려고 몸 안 굽혀도됨 샤워하면서 칙칙 뿌리고 발로 비비면 개꿀깍 드럭스토어에 팔긴하는데 그러면 정가 주고 사야되니까 대형마트나 트레이더스에서 2개이 5천원에 구매 하시면 더 싸게 구매가능',


In [0]:
X = vectorizer.fit_transform(contents)
vectorizer.get_feature_names()

['000',
 '00학번',
 '01',
 '01년생에게',
 '02',
 '03',
 '04',
 '05',
 '07',
 '08',
 '08학번',
 '09',
 '09학번인데',
 '10',
 '10000원',
 '100살까지',
 '10년생이',
 '10매에',
 '10살이나',
 '10학번',
 '10학번아니제',
 '10학번으로',
 '10학번은',
 '10학번이다',
 '10학번이라고요',
 '10학번이라구요',
 '10학번이라굽쇼',
 '10학번이라니',
 '10학번이세요',
 '10학번이셨어요',
 '10학번이야',
 '10학번이었어요',
 '10학번이요',
 '10학번이유',
 '10학번인데',
 '11',
 '112',
 '12',
 '12000원',
 '12학번',
 '12학번입니다',
 '13',
 '13000원',
 '13학번인',
 '14년인가',
 '15',
 '15000어치',
 '1500원에',
 '15800',
 '15800원',
 '15년도',
 '15년인가',
 '15살때',
 '15학번',
 '15학번부터',
 '15학번에서',
 '15학번이에요',
 '15학번인',
 '15학번인데',
 '15학번입니다',
 '15학번입니다유ㅜㅜ',
 '16',
 '16학번',
 '17',
 '17000원',
 '17500라은드어라운드',
 '17500원',
 '17학번인데',
 '17학번입니다',
 '17학번입니다ㅎㅎㅎ내일',
 '18',
 '18000',
 '18000원',
 '18살이라고',
 '18학번',
 '18학번이',
 '18학번입니당',
 '19500원',
 '19인',
 '19학번이지만',
 '19학번인',
 '19학번하구',
 '1ㄱㅇㄷ필리밀리',
 '1년',
 '1년이고',
 '1년전쯤이었나',
 '1되는',
 '1인데',
 '1일1리뷰한번만',
 '1학년',
 '1학년때',
 '1호',
 '20',
 '2000년대',
 '2019년보다',
 '2020',
 '2020ver도',
 '2020년',
 '2020년도',

In [0]:
X.toarray().transpose()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [0]:
X = vectorizer.fit_transform(contents)
num_samples, num_features = X.shape
num_samples, num_features

(619, 3984)

In [0]:
new_post = ['언니 너무 귀엽고 예뻐요 ㅠㅠㅠㅠㅠ']
new_post_vec = vectorizer.transform(new_post)
new_post_vec.toarray()

array([[0, 0, 0, ..., 0, 0, 0]])

In [0]:
import scipy as sp

def dist_raw(v1, v2):
    delta = v1 - v2
    return sp.linalg.norm(delta.toarray())

In [0]:
best_doc = None
best_dist = 65535
best_i = None

for i in range(0, num_samples):
    post_vec = X.getrow(i)
    d = dist_raw(post_vec, new_post_vec)
    
    print("새로운 댓글 %i with 일치도와 거리=%.2f   : %s" %(i,d,contents[i]))
    
    if d<best_dist:
        best_dist = d
        best_i = i

새로운 댓글 0 with 일치도와 거리=5.29   : ??CC click please??????발렌타이 데이에,, 핸드워시 영상 올리는 내가 일류입니다,,,여러분?? 속으로 노래부르며 30초 손씻기 잊지마유?? 생활밀착형 유튜버니께 생활밀착형 콘텐츠로 찾아뵐게유????
새로운 댓글 1 with 일치도와 거리=4.00   : 손도 깨끗이 씻고좋은 일에도 쓰이고일석 이조네요??여윽시 연두콩님??p.s 연두콩님도 코로나 조심하세요??
새로운 댓글 2 with 일치도와 거리=4.24   : ㅠㅠ올라온거 확인하자마자 사라져서 내 손가락 원망하고 있었는데..다행히 빨리 다시 올라와서 오늘 하루도 행복??
새로운 댓글 3 with 일치도와 거리=3.74   : 생일 축하합니다~ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ손 씻으면서 속으로 신나게 부르고 있을 상상을 하니 너무 귀엽고 신박해 ㅠ??
새로운 댓글 4 with 일치도와 거리=3.00   : 코로나까지 걱정해 주는 언니...?? 언니도 조심하세여
새로운 댓글 5 with 일치도와 거리=4.00   : 2:09 생활공작소 진짜 좋아영!! 다른 제품도 거의다 써봤는데 케이스도 예쁘고 넘 좋음 ??
새로운 댓글 6 with 일치도와 거리=3.61   : 라이브 때 ??생활 밀착형 유투버??가 핸드워시 소개해준다구해서 기다렷구만유~~~~넘 고마워유~~~~??
새로운 댓글 7 with 일치도와 거리=3.87   : 아니 마음도 너무 이쁘고 오늘 얼구도 넘 이쁘고ㅠㅠ 진짜 화장품이랑 옷 정보 너무 궁그매여!!
새로운 댓글 8 with 일치도와 거리=3.00   : 언니 머리 흑발 염색한거에여 아니면 원래 머리에용?? 예뻐요 ????
새로운 댓글 9 with 일치도와 거리=2.83   : 두콩언니 얼굴 무슨일이예여??? 너무 존예잖아요!!!!!!
새로운 댓글 10 with 일치도와 거리=4.58   : 거의 끝까지 다 봤는데 그래도 다시 볼께요^^이번주 주말에 날씨가 비가온뒤에 일요일에는 눈이 온다고 하니 길 다니실때

In [0]:
from konlpy.tag import Okt
t = Okt()

In [0]:
contents_tokens = [t.morphs(row) for row in contents]
contents_tokens

[['??',
  'CC',
  'click',
  'please',
  '??????',
  '발렌',
  '타이',
  '데이',
  '에',
  ',,',
  '핸드',
  '워시',
  '영상',
  '올리는',
  '내',
  '가',
  '일류',
  '입니다',
  ',,,',
  '여러분',
  '??',
  '속',
  '으로',
  '노래',
  '부르며',
  '30초',
  '손씻기',
  '잊지',
  '마유',
  '??',
  '생활',
  '밀착',
  '형',
  '유튜버니',
  '께',
  '생활',
  '밀착',
  '형',
  '콘텐츠',
  '로',
  '찾아',
  '뵐게',
  '유',
  '????'],
 ['손',
  '도',
  '깨끗이',
  '씻고',
  '좋은',
  '일',
  '에도',
  '쓰이',
  '고일석',
  '이조',
  '네',
  '요',
  '??',
  '여윽시',
  '연',
  '두',
  '콩',
  '님',
  '??',
  'p',
  '.',
  's',
  '연',
  '두',
  '콩',
  '님',
  '도',
  '코로나',
  '조심하세요',
  '??'],
 ['ㅠㅠ',
  '올라온거',
  '확인',
  '하',
  '자마자',
  '사라져서',
  '내',
  '손가락',
  '원망하고',
  '있었는데',
  '..',
  '다행히',
  '빨리',
  '다시',
  '올라와서',
  '오늘',
  '하루',
  '도',
  '행복',
  '??'],
 ['생일',
  '축하',
  '합니다',
  '~',
  'ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ',
  '손',
  '씻으면서',
  '속',
  '으로',
  '신나게',
  '부르고',
  '있을',
  '상상',
  '을',
  '하니',
  '너무',
  '귀엽고',
  '신',
  '박해',
  'ㅠ',
  '??'],
 ['코로나', '까지', '걱정', '해', '주는', '언니'

# 문장의 유사도를 측정하는 방법

TFIDF 를 통한 문장의 유사도를 측정

TF-IDF(Term Frequency - Inverse Document Frequency)는 정보 검색과 텍스트 마이닝에서 이용하는 가중치로, 여러 문서로 이루어진 문서군이 있을 때 어떤 단어가 특정 문서 내에서 얼마나 중요한 것인지를 나타내는 통계적 수치이다. 문서의 핵심어를 추출하거나, 검색 엔진에서 검색 결과의 순위를 결정하거나, 문서들 사이의 비슷한 정도를 구하는 등의 용도로 사용할 수 있다.

TF(단어 빈도, term frequency)는 특정한 단어가 문서 내에 얼마나 자주 등장하는지를 나타내는 값으로, 이 값이 높을수록 문서에서 중요하다고 생각할 수 있다. 하지만 단어 자체가 문서군 내에서 자주 사용되는 경우, 이것은 그 단어가 흔하게 등장한다는 것을 의미한다. 이것을 DF(문서 빈도, document frequency)라고 하며, 이 값의 역수를 IDF(역문서 빈도, inverse document frequency)라고 한다. TF-IDF는 TF와 IDF를 곱한 값이다.

IDF 값은 문서군의 성격에 따라 결정된다. 예를 들어 '원자'라는 낱말은 일반적인 문서들 사이에서는 잘 나오지 않기 때문에 IDF 값이 높아지고 문서의 핵심어가 될 수 있지만, 원자에 대한 문서를 모아놓은 문서군의 경우 이 낱말은 상투어가 되어 각 문서들을 세분화하여 구분할 수 있는 다른 낱말들이 높은 가중치를 얻게 된다.

In [0]:
contents_for_vectorize = []

for content in contents_tokens:
    sentence = ''
    for word in content:
        sentence = sentence + ' ' + word
        
    contents_for_vectorize.append(sentence)
    
contents_for_vectorize

[' ?? CC click please ?????? 발렌 타이 데이 에 ,, 핸드 워시 영상 올리는 내 가 일류 입니다 ,,, 여러분 ?? 속 으로 노래 부르며 30초 손씻기 잊지 마유 ?? 생활 밀착 형 유튜버니 께 생활 밀착 형 콘텐츠 로 찾아 뵐게 유 ????',
 ' 손 도 깨끗이 씻고 좋은 일 에도 쓰이 고일석 이조 네 요 ?? 여윽시 연 두 콩 님 ?? p . s 연 두 콩 님 도 코로나 조심하세요 ??',
 ' ㅠㅠ 올라온거 확인 하 자마자 사라져서 내 손가락 원망하고 있었는데 .. 다행히 빨리 다시 올라와서 오늘 하루 도 행복 ??',
 ' 생일 축하 합니다 ~ ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ 손 씻으면서 속 으로 신나게 부르고 있을 상상 을 하니 너무 귀엽고 신 박해 ㅠ ??',
 ' 코로나 까지 걱정 해 주는 언니 ...?? 언니 도 조심하세여',
 ' 2:09 생활 공작 소 진짜 좋 아영 !! 다른 제품 도 거의 다 써 봤는데 케이스 도 예쁘고 넘 좋음 ??',
 ' 라이브 때 ?? 생활 밀착 형 유투 버 ?? 가 핸드 워시 소개 해준다 구해 서 기 다 렷 구 만유 ~~~~ 넘 고마워 유 ~~~~??',
 ' 아니 마음 도 너무 이쁘고 오늘 얼구도 넘 이쁘고 ㅠㅠ 진짜 화장품 이랑 옷 정보 너무 궁 그 매 여 !!',
 ' 언니 머리 흑발 염색 한 거 에여 아니면 원래 머리 에 용 ?? 예뻐요 ????',
 ' 두 콩 언니 얼굴 무슨 일이 예여 ??? 너무 존예 잖아요 !!!!!!',
 ' 거의 끝 까지 다 봤는데 그래도 다시 볼께요 ^^ 이번 주 주말 에 날씨 가 비 가온 뒤 에 일요일 에는 눈 이 온다고 하니 길 다니실 때 조심하세요 ^^',
 ' 비행기 에서 빵터졌지 뭐 에 유 ㅋㅋㅋ 이런 꿀 정보 감사해 유 증말루 두 콩 님 두 코로나 조심하세 유 ~~',
 ' 유익한 영상 항상 감사히 봐요 ?? 감사함은 댓글 로 종종 갚을게요 ????',
 ' 이 언니 ?? 너무 스윗 하고 상큼 하잖 아유 .. 첫 멘트 로 그냥

In [0]:
X = vectorizer.fit_transform(contents_for_vectorize)
num_samples, num_features = X.shape
num_samples, num_features

(619, 2969)

In [0]:
vectorizer.get_feature_names()

['00',
 '000',
 '01',
 '01년',
 '02',
 '03',
 '04',
 '05',
 '07',
 '08',
 '09',
 '10',
 '100',
 '10000원',
 '10년',
 '11',
 '112',
 '12',
 '12000원',
 '13',
 '13000원',
 '14년',
 '15',
 '15000',
 '1500원',
 '15800',
 '15800원',
 '15년',
 '16',
 '17',
 '17000원',
 '17500',
 '17500원',
 '18',
 '18000',
 '18000원',
 '19',
 '19500원',
 '1년',
 '1일',
 '20',
 '2000년',
 '2019년',
 '2020',
 '2020년',
 '2022',
 '2023년',
 '2025년',
 '20분',
 '21',
 '21500원',
 '21분',
 '22',
 '24',
 '2400',
 '24000원',
 '24년',
 '25',
 '26',
 '27000',
 '27000원',
 '2800원',
 '2년',
 '2월',
 '30',
 '30000원',
 '30초',
 '32000원',
 '33',
 '3500',
 '35000원',
 '3500원',
 '35만',
 '36000원',
 '365',
 '38',
 '3800원',
 '39',
 '3만원',
 '40',
 '41',
 '42',
 '43',
 '46',
 '49',
 '4년',
 '50',
 '500',
 '51',
 '54',
 '56',
 '57',
 '5년',
 '5천원',
 '5초',
 '60만',
 '6년',
 '70만',
 '7900원',
 '80',
 '822',
 '850',
 '851',
 '88',
 '880',
 '91년',
 '9900원',
 'alwayssss',
 'anything',
 'asmr',
 'beautiful',
 'but',
 'cc',
 'click',
 'cute',
 'don',
 'forever',
 'hello'

In [0]:
X.toarray().transpose()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [0]:
new_post = ['언니 꿀템 소개 감사해요! 꼭 살게요 항상 잘보고 있어요']
new_post_tokens = [t.morphs(row) for row in new_post]

new_post_for_vectorize = []

for content in new_post_tokens:
    sentence = ''
    for word in content:
        sentence = sentence + ' ' + word
        
    new_post_for_vectorize.append(sentence)
    
new_post_for_vectorize

[' 언니 꿀템 소개 감사해요 ! 꼭 살게요 항상 잘 보고 있어요']

In [0]:
new_post_vec = vectorizer.transform(new_post_for_vectorize)

new_post_vec.toarray()

array([[0, 0, 0, ..., 0, 0, 0]])

In [0]:
import scipy as sp

def dist_raw(v1, v2):
    delta = v1 - v2
    return sp.linalg.norm(delta.toarray())

In [0]:
best_doc = None
best_dist = 65535
best_i = None

for i in range(0, num_samples):
    post_vec = X.getrow(i)
    d = dist_raw(post_vec, new_post_vec)
    
    print("새로운 댓글 %i with 일치도와 거리=%.2f   : %s" %(i,d,contents[i]))
    
    if d<best_dist:
        best_dist = d
        best_i = i

새로운 댓글 0 with 일치도와 거리=6.24   : ??CC click please??????발렌타이 데이에,, 핸드워시 영상 올리는 내가 일류입니다,,,여러분?? 속으로 노래부르며 30초 손씻기 잊지마유?? 생활밀착형 유튜버니께 생활밀착형 콘텐츠로 찾아뵐게유????
새로운 댓글 1 with 일치도와 거리=4.12   : 손도 깨끗이 씻고좋은 일에도 쓰이고일석 이조네요??여윽시 연두콩님??p.s 연두콩님도 코로나 조심하세요??
새로운 댓글 2 with 일치도와 거리=4.69   : ㅠㅠ올라온거 확인하자마자 사라져서 내 손가락 원망하고 있었는데..다행히 빨리 다시 올라와서 오늘 하루도 행복??
새로운 댓글 3 with 일치도와 거리=4.58   : 생일 축하합니다~ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ손 씻으면서 속으로 신나게 부르고 있을 상상을 하니 너무 귀엽고 신박해 ㅠ??
새로운 댓글 4 with 일치도와 거리=3.46   : 코로나까지 걱정해 주는 언니...?? 언니도 조심하세여
새로운 댓글 5 with 일치도와 거리=4.36   : 2:09 생활공작소 진짜 좋아영!! 다른 제품도 거의다 써봤는데 케이스도 예쁘고 넘 좋음 ??
새로운 댓글 6 with 일치도와 거리=4.00   : 라이브 때 ??생활 밀착형 유투버??가 핸드워시 소개해준다구해서 기다렷구만유~~~~넘 고마워유~~~~??
새로운 댓글 7 with 일치도와 거리=4.90   : 아니 마음도 너무 이쁘고 오늘 얼구도 넘 이쁘고ㅠㅠ 진짜 화장품이랑 옷 정보 너무 궁그매여!!
새로운 댓글 8 with 일치도와 거리=4.00   : 언니 머리 흑발 염색한거에여 아니면 원래 머리에용?? 예뻐요 ????
새로운 댓글 9 with 일치도와 거리=3.61   : 두콩언니 얼굴 무슨일이예여??? 너무 존예잖아요!!!!!!
새로운 댓글 10 with 일치도와 거리=4.80   : 거의 끝까지 다 봤는데 그래도 다시 볼께요^^이번주 주말에 날씨가 비가온뒤에 일요일에는 눈이 온다고 하니 길 다니실때

In [0]:
print("Best post is %i, dist = %.2f" % (best_i, best_dist))
print('-->', new_post)
print('---->', contents[best_i])

Best post is 46, dist = 2.24
--> ['언니 꿀템 소개 감사해요! 꼭 살게요 항상 잘보고 있어요']
----> 꿀템 소개 감사해요!! 언니도 건강 조심하셔요?


In [0]:
for i in range(0,len(contents)):
    print(X.getrow(i).toarray())
    
print('---------------------')
print(new_post_vec.toarray())

[[0 0 0 ... 0 0 0]]
[[0 0 0 ... 0 0 0]]
[[0 0 0 ... 0 0 0]]
[[0 0 0 ... 0 0 0]]
[[0 0 0 ... 0 0 0]]
[[0 0 0 ... 0 0 0]]
[[0 0 0 ... 0 0 0]]
[[0 0 0 ... 0 0 0]]
[[0 0 0 ... 0 0 0]]
[[0 0 0 ... 0 0 0]]
[[0 0 0 ... 0 0 0]]
[[0 0 0 ... 0 0 0]]
[[0 0 0 ... 0 0 0]]
[[0 0 0 ... 0 0 0]]
[[0 0 0 ... 0 0 0]]
[[0 0 0 ... 0 0 0]]
[[0 0 0 ... 0 0 0]]
[[0 0 0 ... 0 0 0]]
[[0 0 0 ... 0 0 0]]
[[0 0 0 ... 0 0 0]]
[[0 0 0 ... 0 0 0]]
[[0 0 0 ... 0 0 0]]
[[0 0 0 ... 0 0 0]]
[[0 0 0 ... 0 0 0]]
[[0 0 0 ... 0 0 0]]
[[0 0 0 ... 0 0 0]]
[[0 0 0 ... 0 0 0]]
[[0 0 0 ... 0 0 0]]
[[0 0 0 ... 0 0 0]]
[[0 0 0 ... 0 0 0]]
[[0 0 0 ... 0 0 0]]
[[0 0 0 ... 0 0 0]]
[[0 0 0 ... 0 0 0]]
[[0 0 0 ... 0 0 0]]
[[0 0 0 ... 0 0 0]]
[[0 0 0 ... 0 0 0]]
[[0 0 0 ... 0 0 0]]
[[0 0 0 ... 0 0 0]]
[[0 0 0 ... 0 0 0]]
[[0 0 0 ... 0 0 0]]
[[0 0 0 ... 0 0 0]]
[[0 0 0 ... 0 0 0]]
[[0 0 0 ... 0 0 0]]
[[0 0 0 ... 0 0 0]]
[[0 0 0 ... 0 0 0]]
[[0 0 0 ... 0 0 0]]
[[0 0 0 ... 0 0 0]]
[[0 0 0 ... 0 0 0]]
[[0 0 0 ... 0 0 0]]
[[0 0 0 ... 0 0 0]]


In [0]:
def dist_norm(v1, v2):
    v1_normalized = v1 / sp.linalg.norm(v1.toarray())
    v2_normalized = v2 / sp.linalg.norm(v2.toarray())
    
    delta = v1_normalized - v2_normalized
    
    return sp.linalg.norm(delta.toarray())

In [0]:
best_doc = None
best_dist = 65535
best_i = None

for i in range(0, num_samples):
    post_vec = X.getrow(i)
    d = dist_norm(post_vec, new_post_vec)
    
    print("정규화 댓글 %i with 일치도와 거리=%.2f   : %s" %(i,d,contents[i]))
    
    if d<best_dist:
        best_dist = d
        best_i = i

정규화 댓글 0 with 일치도와 거리=1.41   : ??CC click please??????발렌타이 데이에,, 핸드워시 영상 올리는 내가 일류입니다,,,여러분?? 속으로 노래부르며 30초 손씻기 잊지마유?? 생활밀착형 유튜버니께 생활밀착형 콘텐츠로 찾아뵐게유????
정규화 댓글 1 with 일치도와 거리=1.41   : 손도 깨끗이 씻고좋은 일에도 쓰이고일석 이조네요??여윽시 연두콩님??p.s 연두콩님도 코로나 조심하세요??
정규화 댓글 2 with 일치도와 거리=1.41   : ㅠㅠ올라온거 확인하자마자 사라져서 내 손가락 원망하고 있었는데..다행히 빨리 다시 올라와서 오늘 하루도 행복??
정규화 댓글 3 with 일치도와 거리=1.41   : 생일 축하합니다~ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ손 씻으면서 속으로 신나게 부르고 있을 상상을 하니 너무 귀엽고 신박해 ㅠ??
정규화 댓글 4 with 일치도와 거리=1.22   : 코로나까지 걱정해 주는 언니...?? 언니도 조심하세여
정규화 댓글 5 with 일치도와 거리=1.41   : 2:09 생활공작소 진짜 좋아영!! 다른 제품도 거의다 써봤는데 케이스도 예쁘고 넘 좋음 ??
정규화 댓글 6 with 일치도와 거리=1.33   : 라이브 때 ??생활 밀착형 유투버??가 핸드워시 소개해준다구해서 기다렷구만유~~~~넘 고마워유~~~~??
정규화 댓글 7 with 일치도와 거리=1.41   : 아니 마음도 너무 이쁘고 오늘 얼구도 넘 이쁘고ㅠㅠ 진짜 화장품이랑 옷 정보 너무 궁그매여!!
정규화 댓글 8 with 일치도와 거리=1.33   : 언니 머리 흑발 염색한거에여 아니면 원래 머리에용?? 예뻐요 ????
정규화 댓글 9 with 일치도와 거리=1.32   : 두콩언니 얼굴 무슨일이예여??? 너무 존예잖아요!!!!!!
정규화 댓글 10 with 일치도와 거리=1.41   : 거의 끝까지 다 봤는데 그래도 다시 볼께요^^이번주 주말에 날씨가 비가온뒤에 일요일에는 눈이 온다고 하니 길 다니실때

/usr/local/lib/python3.6/dist-packages/scipy/sparse/base.py:585: RuntimeWarning: divide by zero encountered in double_scalars
  return self.astype(np.float_)._mul_scalar(1./other)


정규화 댓글 176 with 일치도와 거리=1.41   : 이 영상에서 한 화장법 영상찍어주세요오 ㅜㅜ
정규화 댓글 177 with 일치도와 거리=1.20   : 우리 연두부 대학때 다 키운건 우리 두콩 언니다.... 나 울언니 제품보고 다 사다가 책상넘쳐서 미친다..
정규화 댓글 178 with 일치도와 거리=1.41   : 저거 핸드크림 진짜좋아용ㅠㅠ저도 쓰고 있는데 간호실습 나가고 있는데 손이 너무 건조해져서 손닦고 쓸때마다 끈적임도 덜하고 냄새도 좋아용 ㅠㅠ
정규화 댓글 179 with 일치도와 거리=1.27   : 판검사 되려고 법학과 들어가서 언니 영상 보고 코덕되어 뷰티마케터가 된 15학번이에요...?? 연두콩님 제가 5년째 많이 애낍니다??
정규화 댓글 180 with 일치도와 거리=1.41   : 아 영상 보는 내내 귀여워서 입꼬리가 안내려가유ㅠㅠㅠㅠ 진짜 덕분에 새내기는 아니지만 올해 메이크업 제품들은 해결 났슈~~가성비 갑이구 넘 좋네유 살앙해유??
정규화 댓글 181 with 일치도와 거리=1.32   : 엏 진짜 이번 영상 진짜 너무 좋네요! 메이크업 순서부터 지우는 것까지 다 소개 시켜주시다니 대박이네요특히 핸드크림 진짜 향 좋아요 들꽃 향기가 확나요 매일매일 쓰고 있어요! 세일해서 2800원이니까 한번 사보세요ㅠㅠ????
정규화 댓글 182 with 일치도와 거리=1.24   : 새내기아닌데 와서 보고가유~ ㅋㅋㅋㅋㅋㅋ역시전문유투버 두콩언니 최고~
정규화 댓글 183 with 일치도와 거리=1.41   : 전 이제 내년에 새내기에여ㅠㅠㅠㅠ 1년 기다렸다가 또 보러올께여ㅠㅠㅠ
정규화 댓글 184 with 일치도와 거리=1.41   : 15학번입니다..이언니 믿고갑시다..-5학년..
정규화 댓글 185 with 일치도와 거리=1.24   : 안녕하세유~~ 언니가 대학을 보낸 18학번 연두부여유~~ 언니가 18학번이 새내기 일 때 추천해줬던 제품들 다 사서 써봤고 그 중에 아직까지 쓰는 제품들도 있어요 !! 두콩언니의 새내기 추천템은 믿고

In [0]:
print("가장 일치도가 높은 정규화된 문장번호  %i, 핵심 단어와 떨어진 거리 = %.2f\n\n\n" % (best_i, best_dist))
print('대상 댓글-->', new_post)
print('모델이 학습하고 비슷하다 여기는 댓글-->', contents[best_i])

가장 일치도가 높은 정규화된 문장번호  46, 핵심 단어와 떨어진 거리 = 0.87



대상 댓글--> ['언니 꿀템 소개 감사해요! 꼭 살게요 항상 잘보고 있어요']
모델이 학습하고 비슷하다 여기는 댓글--> 꿀템 소개 감사해요!! 언니도 건강 조심하셔요?
